In [1]:
import pandas as pd

/var/folders/0p/t55f5t150hv1sqqwhrm_md340000gn/T/ipykernel_28258/2035458946.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [49]:
# Assuming your CSV file is named 'your_data.csv'
df = pd.read_csv('COMPLETE_player_data.csv')

# Group rows based on 'Opp' and 'Position' columns
grouped_df = df.groupby(['Opp', 'Position_Filter']).agg(list).reset_index()

In [50]:
grouped_df.to_csv('grouped_data.csv', index=False)


In [45]:
import numpy as np

In [51]:
# Define a custom function to calculate the average of a list excluding the largest and smallest values
def calculate_row_average(row, column_name, exclude_largest=1, exclude_smallest=1):
    sorted_values = np.sort(row[column_name])[exclude_smallest:-exclude_largest]
    return np.mean(sorted_values)

# Define a custom function to calculate the standard deviation of a list excluding the largest and smallest values
def calculate_row_stdev(row, column_name, exclude_largest=1, exclude_smallest=1):
    sorted_values = np.sort(row[column_name])[exclude_smallest:-exclude_largest]
    return np.std(sorted_values)

In [52]:


# Apply the custom function to each row to create new columns for each attribute
attributes_to_calculate = ['Points_Diff', 'Rebounds_Diff', 'Assists_Diff', 'Steals_Diff', 'Blocks_Diff']

for attribute in attributes_to_calculate:
    grouped_df[f'{attribute}_Avg'] = grouped_df.apply(
        lambda row: calculate_row_average(row, column_name=attribute, exclude_largest=1, exclude_smallest=1), axis=1
    )

    grouped_df[f'{attribute}_Std'] = grouped_df.apply(
        lambda row: calculate_row_stdev(row, column_name=attribute, exclude_largest=1, exclude_smallest=1), axis=1
    )



In [53]:
# Select specific columns for the output CSV
output_columns = ['Opp', 'Position_Filter', 'Points_Diff_Avg', 'Points_Diff_Std', 'Rebounds_Diff_Avg', 'Rebounds_Diff_Std', 'Assists_Diff_Avg', 'Assists_Diff_Std', 'Steals_Diff_Avg', 'Steals_Diff_Std','Blocks_Diff_Avg', 'Blocks_Diff_Std']

# Save the DataFrame with the selected columns to a new CSV file
grouped_df[output_columns].to_csv('averages_grouped.csv', index=False)


In [54]:
import pandas as pd

In [60]:
# Assuming your CSV file is named 'your_file.csv'
df = pd.read_csv('averages_grouped.csv')

# List of column prefixes (e.g., 'Points_Diff', 'Rebounds_Diff', etc.)
column_prefixes = ['Points_Diff', 'Rebounds_Diff', 'Assists_Diff', 'Steals_Diff', 'Blocks_Diff']

# Add a new column for standard deviation divided by mean for each prefix
for prefix in column_prefixes:
    avg_column = f'{prefix}_Avg'
    std_column = f'{prefix}_Std'

    # Calculate new value: std / mean
    new_value_column = f'{prefix}_Cv'
    df[new_value_column] = df[std_column] / df[avg_column]


# Reorder columns to have Avg, Std, and New_Value together for each category
new_column_order = ["Opp", "Position_Filter"]
for prefix in column_prefixes:
    avg_column = f'{prefix}_Avg'
    std_column = f'{prefix}_Std'
    new_value_column = f'{prefix}_Cv'

    new_column_order.extend([avg_column, std_column, new_value_column])

df = df[new_column_order]

# Save the updated DataFrame to a new CSV file if needed
df.to_csv('summary_avg_std_cv.csv', index=False)
